# Set up

In [1]:
import torch
import gpytorch
import pandas as pd
import numpy as np
import tqdm as tqdm


import os
import pickle

import GP_functions.Loss_function as Loss_function
import GP_functions.bound as bound
import GP_functions.Estimation as Estimation
import GP_functions.Training as Training
import GP_functions.Prediction as Prediction
import GP_functions.GP_models as GP_models
import GP_functions.Tools as Tools
import GP_functions.FeatureE as FeatureE

# data

In [2]:
X_train = pd.read_csv('Data/X_train.csv', header=None, delimiter=',').values
X_test = pd.read_csv('LocalDisease/X_1_1.csv', header=None, delimiter=',').values

Y_train_21 = pd.read_csv('LocalDisease/Y_train_std_pca.csv', header=None, delimiter=',').values
Y_test_21 = pd.read_csv('LocalDisease/Y_data_1_1_pca.csv', header=None, delimiter=',').values


train_x = torch.tensor(X_train, dtype=torch.float32)
test_x = torch.tensor(X_test, dtype=torch.float32)

train_y_21 = torch.tensor(Y_train_21, dtype=torch.float32)
test_y_21 = torch.tensor(Y_test_21, dtype=torch.float32)



torch.set_default_dtype(torch.float32)

In [ ]:
X_test

In [ ]:
Y_train_21

In [12]:
Y_test_21

array([[  5.66421631, -19.13963123,  -9.00952994, ...,  -0.71061098,
          0.41942116,   0.09788022],
       [  5.89023632, -19.05614506,  -9.07184596, ...,  -0.84434076,
          0.73948717,  -0.14331645],
       [  4.99109565, -18.26724214,  -7.98400442, ...,  -0.72211728,
          0.31312728,   0.16704077],
       ...,
       [  5.29074328, -17.57189084,  -9.99057076, ...,  -0.88492758,
          0.30007534,   0.12787756],
       [  6.16650671, -20.38138433,  -8.23736032, ...,  -0.42758968,
          0.70314878,  -0.05705013],
       [  5.07618097, -16.52722064, -10.5249159 , ...,  -0.89096639,
          0.31573823,   0.10705541]])

# Emulator

In [3]:
Device = 'cpu'

In [4]:
checkpoint = torch.load('multitask_gp_checkpoint_LocalD.pth', map_location=Device)
model_params = checkpoint['model_params']

MVGP_models = GP_models.MultitaskVariationalGP(train_x, train_y_21, 
                                               num_latents=model_params['num_latents'], 
                                               num_inducing=model_params['num_inducing'], 
                                               covar_type=model_params['covar_type']).to(Device)

MVGP_models.load_state_dict(checkpoint['model_state_dict'])

MVGP_likelihoods = gpytorch.likelihoods.MultitaskGaussianLikelihood(num_tasks=train_y_21.shape[1]).to(Device)
MVGP_likelihoods.load_state_dict(checkpoint['likelihood_state_dict'])

MVGP_models.eval()
MVGP_likelihoods.eval()


C:\Users\r4718\AppData\Local\Temp\ipykernel_30268\1906017165.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('multitask_gp_checkpoint_LocalD.pth'

MultitaskGaussianLikelihood(
  (raw_task_noises_constraint): GreaterThan(1.000E-04)
  (raw_noise_constraint): GreaterThan(1.000E-04)
)

In [5]:
col_min = np.nanmin(Y_train_21, axis=0)
col_max = np.nanmax(Y_train_21, axis=0)


((Y_test_21 >= col_min) & (Y_test_21 <= col_max)).sum(axis=1)

array([17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
       17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17,
       17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 15, 17, 17, 17,
       17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 16, 17, 17, 17, 16, 17, 16,
       17, 17, 17, 16, 17, 17, 17, 17, 16, 17, 16, 17, 17, 17, 17, 17, 16,
       17, 17, 17, 17, 17, 16, 16, 17, 16, 17, 17, 16, 16, 16, 16])

In [6]:
Y_test_21.shape

(100, 21)

# PE

In [7]:
row_idx = 0

input_point = test_y_21[row_idx, :]

local_train_x, local_train_y = Tools.find_k_nearest_neighbors_CPU(input_point, train_x, train_y_21, k=100)

bounds = bound.get_bounds(local_train_x)

In [13]:
preds_tmp = Prediction.preds_for_one_model(
    MVGP_models, MVGP_likelihoods, test_x[row_idx,:].unsqueeze(0).to(Device)
    ).cpu().detach().numpy()

In [23]:
from sklearn.metrics import r2_score

preds_tmp = Prediction.preds_for_one_model(
    MVGP_models, MVGP_likelihoods, test_x.to(Device)
    ).cpu().detach().numpy()


r2_score(preds_tmp.reshape(100, 21), test_y_21.numpy())
np.mean((preds_tmp.reshape(100, 21) - test_y_21.numpy()) ** 2)

5.1514072

In [22]:
test_y_21.numpy().shape

(100, 21)

In [20]:
preds_tmp.reshape(100, 21).shape

(100, 21)

In [ ]:


estimated_params_tmp, _ = Estimation.multi_start_estimation(
    MVGP_models, MVGP_likelihoods, row_idx, test_y_21, bounds,
    Estimation.estimate_params_for_one_model_Adam, num_starts=8, num_iterations=2000, lr=0.01,
    patience=10, attraction_threshold=0.1, repulsion_strength=0.1, device=Device
)